**By Peter A. Stokes, École Pratique des Hautes Études – Université PSL**


_These are brief notes summarising the key points of image analysis and manipulation using Python. They are intended as teaching aids for the course on 'Image Processing with Python' which is part of the École d'été d'éditions numériques / humanités numériques (EDEEN). For more details see https://edeen.sciencesconf.org. They assume a pre-existing directory structure with images of Cambridge, Corpus Christi College MS 44 which can be obtained from Parker on the Web: https://parker.stanford.edu_.

In a very interesting blog post, Jesse Hurlbut proposed different ways of calculating a 'manuscript average', by taking all the images of a complete manuscript and overlaying them on top of each other in different ways. Doing this shows very vividly the consistency (or lack thereof) in layout, and also highlights any specific pages which vary significantly from the norm. You can see the original blog post at http://jessehurlbut.net/wp/mssart/?page_id=2097

The goal of this exercise is to write a program using PIL to create our own 'manuscript average' images using our own manuscripts. We have already learned everything we need to do this in the first exercise; we just need to put it all together.

# Step By Step...

Let's go through the process step by step to begin with, and then we can put it all together at the end. This is always a good approach: to do things one bit at a time, testing along the way to make sure it all works, rather than trying to write everything at once and then having a real mess to sort out when things go wrong.

## Setup and initialisation

Very often, scripts consist of three parts: 

1. An initial part for importing libraries and setting up initial variables;
1. A main central part for doing the 'heavy' processing;
1. A final part for producing output (saving results, printing to the console, etc.)

We can see this pattern in the code here. First, we import the necessary libraries, initialise some variables, and read in the list of files that we will be processing.

In [ ]:
from PIL import Image, ImageChops, ImageOps
import os, glob

directory = 'CCCC 44/'
filelist = glob.glob(directory + '*.jpg')

In order for the code to work properly, all the images need to be the same size. In practice the images are often slightly different sizes, so we will need to scale them. The problem is that we have to decide in advance what size all the images should be. Here, I've decided simply to find the smallest image and to scale all the other images to that. The procedure we follow is this:

1. Set the initial minimum size to zero
1. For each image file in the directory:
   1. If the minimum size is still zero then set the minimum to the size of this image.
   1. Otherwise, if the current image is smaller than the minimum size then save the current size as the minimum
   1. Otherwise, if this image is bigger than the smallest so far, then ignore this image and go on to the next one.
   
The last two 'otherwise' can be rewritten as follows:

1. Set the initial minimum size to zero
1. For each image file in the directory:
   1. If the minimum size is still zero then set the minimum to the size of this image.
   1. Otherwise, set the current minimum to either the value it has now or to the size of the new image, whichever is smaller.
   
This is harder for us to understand but is a bit easier for the computer.

In [ ]:
minx = 0
miny = 0

for f in filelist:
    im = Image.open(f)
        
    if (minx, miny) == (0,0):
        minx, miny = im.size
    else:
        x, y = im.size
        minx = min(x, minx)
        miny = min(y, miny)

Now we need to set up the images. We create two 'empty' images to which we will add all the images from our manuscript. The one for blending images should be entirely black to begin with, and the one for the `darker` operator should be black.

We also need to decide what transparency value to use for the images. Ideally it makes sense to use an equal transparency for each image, with the total transparency at 100%. This means that if there are two images then each image should be 50% transparent; four images should be 25% and so on, so n images should have transparency of 1/n. There's a small problem here in that if the transparency is too small then the images seem to come out as black, so we have to set a minimum transparencey as well. A value of `0.01` seems to work, but feel free to try different values here.

In [ ]:
alpha = max(1.0/len(filelist), 0.01)
    
im_blend = Image.new('RGB', (minx, miny))
im_dark = Image.new('RGB', (minx, miny), 'white')

## Processing the images

Now that we have set everything up, we can start processing the images. To begin with, let's figure out the process for a single image, and then we can put it all together into a loop.

The first thing to do is to load in the image and resize it:

In [ ]:
f = directory + '001.jpg'
im = Image.open(f)

im_rz = im.resize((minx, miny))

We then need to see if the image is of a recto or a verso page. Normally, rectos and versos are mirrors of each other in terms of page layout (i.e. the left margin on a recto will match the right margin on a verso, and so on). For this reason we need to mirror all the recto pages (or all the versos) to make sure they lign up. In order to do this, we assume that the file name includes the necessary information:

1. If the file name (excluding the extension) ends in 'v' then we assume it's a verso (e.g. 002v, 35v, etc.);
1. If the file name ends in 'r' then we assume it's a recto (e.g. 215r, etc.)
1. If the file name comprises a number, then we assume it refers to a page, and so:
   1. If the number is odd then we assume it's a recto
   1. If the number is even then we assume it's a verso

In order to tell if a number is even or odd, we can use the _modulo_ operator, %. The modulo operator gives the remainder if the first number is divided by the second number. Therefore n % 2 will be zero if n is even, and will be one if n is odd. 

In [ ]:
# Get the basename without extension
fol = os.path.splitext(os.path.basename(f))[0]

# We need to flip all the versos (or all the rectos)
# so that the page layout matches.
if (fol.lower()[-1] == 'v'):
    imfinal = ImageOps.mirror(im_rz)
elif (fol.lower()[-1] == 'r'):
    imfinal = im_rz
elif (int(fol) % 2 == 0):
    imfinal = ImageOps.mirror(im_rz)
else:
    imfinal = im_rz
    
im_rz.show()

We can now combine the images. Like Jesse Hurlbut, we can use two different methods: `Image.blend` to blend the different images equally together, and `Image.darker` to preserve the darkest part of each image. We won't see much at this point, of course, because we only have one image, but we can have a look anyway just to make sure that something happened.

In [ ]:
im_blend = Image.blend(im_blend, imfinal, alpha)
im_dark = ImageChops.darker(im_dark, imfinal)

im_blend.show()
im_dark.show()

Now we want to put all of this together into a single loop. Notice that this follows the same pattern as we used for automatic batch processing of a set of images in Exercice 1.

A loop like this might take a long time to run, so let's add some `print()` statements to tell the user that things are happening, and also because if there is a problem then this can help us understand where things went wrong.

In [ ]:
for f in filelist:
    print('Reading image', f)
    im = Image.open(f)

    im_rz = im.resize((minx, miny))
    
    # Get the basename without extension
    fol = os.path.splitext(os.path.basename(f))[0]

    # We need to flip all the versos (or all the rectos)
    # so that the page layout matches.
    if (fol.lower()[-1] == 'v'):
        imfinal = ImageOps.mirror(im_rz)
    elif (fol.lower()[-1] == 'r'):
        imfinal = im_rz
    elif (int(fol) % 2 == 0):
        imfinal = ImageOps.mirror(im_rz)
    else:
        imfinal = im_rz
        
    im_blend = Image.blend(im_blend, imfinal, alpha)
    im_dark = ImageChops.darker(im_dark, imfinal)

## The Output

After this we now have our two 'manuscript average' images: one using the blending method and the other using the 'darker' method. All we need to do now is save the images.

It turns out also that the 'blend' image is often very faint. This could probably be fixed by playing with the alpha values above, but we can also use PIL's automatic contrast adjust instead. 

In [ ]:
im_blend = ImageOps.autocontrast(im_blend)

im_blend.save(directory + 'image_blend.png')
im_dark.save(directory + 'image_dark.png')

im_blend.show()
im_dark.show()

# Putting it All Together

That's it! Those are all the steps we need. If we want then we can now put it all together into a single piece of code which you can copy and paste into a stand-alone script to run yourself if you want. Have fun making manuscript averages with your own images, and do also try changing and improving the code according to your own interests and needs.

In [ ]:
"""
Basic script to create 'manuscript average' pages, based on discussion by Jesse
Hurlbut at http://jessehurlbut.net/wp/mssart/?page_id=2097.

This version assumes a directory of JPG images specified in the directory
variable below.

Created on Wed May 16 14:05:04 2018

@author: Peter A. Stokes, École Pratique des Hautes Études – Université PSL
"""


from PIL import Image, ImageChops, ImageOps
import os, glob

directory = 'CCCC 44/'
filelist = glob.glob(directory + '*.jpg')

minx = 0
miny = 0

# First find the smallest image to resize to
print('Reading image sizes...')
for f in filelist:
    im = Image.open(f)
        
    if (minx, miny) == (0,0):
        minx, miny = im.size
    else:
        x, y = im.size
        minx = min(x, minx)
        miny = min(y, miny)
        
# Precalculate alpha to bring it out of the inner loop below
# The alpha seems to have a minimum value below which the images are black
alpha = max(1.0/len(filelist), 0.01)
    
im_blend = Image.new('RGB', (minx, miny))
im_dark = Image.new('RGB', (minx, miny), 'white')

for f in filelist:
    print('Reading image', f)
    im = Image.open(f)

    im_rz = im.resize((minx, miny))
    
    # Get the basename without extension
    fol = os.path.splitext(os.path.basename(f))[0]

    # We need to flip all the versos (or all the rectos)
    # so that the page layout matches.
    if (fol.lower()[-1] == 'v'):
        # Profiling suggests that ImageOps.mirror is slightly faster than Image.transpose
        imfinal = ImageOps.mirror(im_rz)
    elif (fol.lower()[-1] == 'r'):
        imfinal = im_rz
    elif (int(fol) % 2 == 0):
        imfinal = ImageOps.mirror(im_rz)
    else:
        imfinal = im_rz
        
    im_blend = Image.blend(im_blend, imfinal, alpha)
    im_dark = ImageChops.darker(im_dark, imfinal)

# The blended image tends to be very faint, so let's enhance the contrast
im_blend = ImageOps.autocontrast(im_blend)

im_blend.save(directory + 'image_blend.png')
im_dark.save(directory + 'image_dark.png')

im_blend.show()
im_dark.show()

---
![Licence Creative Commons](https://i.creativecommons.org/l/by/4.0/88x31.png)
This work (the contents of this Jupyter Python notebook) is licenced under a [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)